In [1]:
import sys
sys.path.append('./../../')

import ikamoana.fisherieseffort.fisherieseffort as fe
import ikamoana.feedinghabitat.feedinghabitat as fh
import xarray as xr
import pandas as pd
import numpy as np
from typing import Union

In [2]:
filepath = "./../../data/SEAPODYM_data/run-test/data/fisheries/skj_fisheries_data_2021_Nov17.txt"

---
## Fishery Effort Time Coords

In [3]:
fisheries_df = fe.readFile(filepath, skiprows=0)
fisheries_dict = fe.separateFisheries(fisheries_df)
fisheries_rescaled_dict = fe.rescaleFisheries({15:fisheries_dict[15]})
fisheries_groub_by_dict = fe.groupByFisheries(fisheries_rescaled_dict)
f15 = fisheries_groub_by_dict[15]
fe15, _ = fe.fisheriesToDataSet(fisheries_groub_by_dict)
fe15 = fe15[15]
fe15_time = fe15.time.data
print(fe15_time.size)
print(fe15_time[0])
print(fe15_time[-1])

356
1974-01-15T00:00:00.000000000
2014-08-15T00:00:00.000000000


In [3]:
effort_table = pd.read_table(filepath).rename({"yr":"year","mm":"month","dd":"day"},axis=1)
effort_table['time'] = pd.to_datetime(effort_table[["year","month","day"]])
effort_table.drop(['year','month','day'], axis=1)
#effort_table[["f","date","lat","lon","res","E","C","gr"]]
effort_table = effort_table[["f","time","lat","lon","E"]]
effort_table = effort_table.groupby(["f","time","lat","lon"]).sum()

effort_dataset = xr.Dataset.from_dataframe(effort_table)
effort_dataset = xr.Dataset(
    {str(i.data) : effort_dataset.sel(f=i).reset_coords('f')['E']
     for i in effort_dataset.f})
effort_dataset

<xarray.Dataset>
Dimensions:  (time: 858, lat: 98, lon: 180)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-15 1950-02-15 ... 2021-06-15
  * lat      (lat) float64 -49.5 -46.5 -45.5 -44.5 -43.5 ... 49.5 50.5 52.5 55.5
  * lon      (lon) float64 0.5 110.5 111.5 112.5 ... 285.5 286.5 287.5 288.5
Data variables: (12/13)
    1        (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    2        (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    3        (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    4        (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    5        (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    6        (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    ...       ...
    8        (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    9        (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    10       (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    12       (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    13       (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    15       (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan

In [22]:
effort_dataset = effort_dataset.sel(time=effort_dataset.time[200:400])

In [23]:
effort_dataset.sum()

<xarray.Dataset>
Dimensions:  ()
Data variables: (12/13)
    1        float64 6.589e+05
    2        float64 2.46e+05
    3        float64 1.705e+05
    4        float64 2.19e+04
    5        float64 2.196e+04
    6        float64 2.966e+04
    ...       ...
    8        float64 6.018e+07
    9        float64 1.832e+04
    10       float64 3.848e+04
    12       float64 7.871e+04
    13       float64 1.124e+05
    15       float64 1.193e+04

In [24]:
fe.normalizeCoords(effort_dataset,7,1).sum()

<xarray.Dataset>
Dimensions:  ()
Data variables: (12/13)
    1        float64 1.048e+05
    2        float64 3.793e+04
    3        float64 2.373e+04
    4        float64 4.235e+03
    5        float64 3.935e+03
    6        float64 4.272e+03
    ...       ...
    8        float64 9.325e+06
    9        float64 2.749e+03
    10       float64 6.042e+03
    12       float64 1.287e+04
    13       float64 1.922e+04
    15       float64 1.686e+03

---
## Feeding Habitat Time Coords

In [7]:
fh_filepath = "./../../data/SEAPODYM_data/run-test/skj_CLT_e5_3_est.xml"
fh15 = fh.FeedingHabitat(fh_filepath)
fhcoords15_time = fh15.data_structure.coords['time'].data
print(fhcoords15_time.size)
print(fhcoords15_time[0])
print(fhcoords15_time[-1])

384
1979-01-15T00:00:00.000000000
2010-12-16T00:00:00.000000000


---

## But coords are not continuous, we need to fill in the gaps

### Before reshaping

In [8]:
print(fe15.shape)
print(fe15.sum().data)

(356, 50, 54)
48865.0


### After reshaping

In [9]:
tmp = fe._completeCoords(fe15, 30, 1)
print(tmp.shape)
print(tmp.sum().data)

(488, 51, 55)
48865.0


> ### Becarfull : Can't be use to downscale data. ( => Loss )

In [10]:
tmp = fe._completeCoords(fe15, 7, 2)
print(tmp.shape)
print(tmp.sum().data)

(2119, 26, 28)
2408.0


---

In [14]:
fhcoords15_time.size

384

In [16]:
fh15.data_structure.coords['lat'].data.size

60

In [17]:
fh15.data_structure.coords['lon'].data.size

101

In [18]:
fe15

<xarray.DataArray 15 (time: 356, lat: 50, lon: 54)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time     (time) datetime64[ns] 1974-01-15 1974-03-15 ... 2014-08-15
  * lat      (lat) float64 -17.5 -16.5 -15.5 -14.5 -13.5 ... 29.5 30.5 31.5 32.5
  * lon      (lon) float64 212.5 213.5 214.5 215.5 ... 262.5 264.5 265.5 266.5
Attributes:
    gr:       ['P']
    res:      [5]